In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import CM4Xutils
CM4Xutils.__version__

'0.5.0'

In [3]:
import warnings
import dask
import xarray as xr
import xwmt
import xgcm
import matplotlib.pyplot as plt
import numpy as np
import cftime

import doralite
import gfdl_utils.core as gu

In [4]:
grid = CM4Xutils.load_wmt_grid(
    "CM4Xp25",
    interval=str(2000),
    dmget=True
)

Loading CM4Xp25-piControl for interval `2000`.
Issuing dmget command to migrate data to disk. Migration complete.


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


Issuing dmget command to migrate data to disk. Migration complete.


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg
/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Loading CM4Xp25-historical for interval `2000`.
Issuing dmget command to migrate data to disk. Migration complete.


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


Issuing dmget command to migrate data to disk. Migration complete.


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg
/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/xgcm/grid_ufunc.py:836: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Overriding CM4Xp25 grid coordinates from supergrid.
Regridding ice
Inferring Z grid coordinate: depth `z_`


/work/hfd/.conda/envs/CM4X/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [5]:
grid._ds.sigma2.attrs = {
    "long_name": "Potential Density referenced to 2000 dbar (minus 1000 kg/m3)",
    "units": "kg m-3",
    "cell_methods": "area:mean z_l:mean yh:mean xh:mean time:mean",
    "volume": "volcello",
    "area": "areacello",
    "time_avg_info": "average_T1,average_T2,average_DT",
    "description": "Computed offline using the gsw python package implementation of TEOS10.",
}

## Compute the annual mean for the first year

In [6]:
drop_vars = ["average_DT", "average_T1", "average_T2", "vo", "uo"]
ds = grid._ds.sel(exp="forced").isel(time=np.arange(0,12,1), time_bounds=np.arange(0,13,1)).drop_vars(drop_vars).fillna(0.)

In [7]:
def to_numeric_time(date):
    # Convert to number of days since 0001-01-01
    return cftime.date2num(date, units='days since 0001-01-01', calendar='noleap')

# Apply the function to your time array
numeric_time = np.vectorize(to_numeric_time)(ds.time_bounds.values)

# Convert days to seconds
seconds_array = numeric_time * 24 * 60 * 60

ds = ds.assign_coords({"time_delta":xr.DataArray(np.diff(seconds_array), dims=("time",))})

In [8]:
# Annual-mean, weighted by length of each month
# Note: da.weighted behaves strangely when there are NaNs. Fill the NaNs for correct behavior!
ds_mean = ds.drop_dims("time_bounds").weighted(ds.time_delta).mean(dim="time", keep_attrs=True)
ds_mean = ds_mean.expand_dims(["time"]).assign_coords({"time": xr.DataArray(ds.time_bounds[[6]].values, dims=("time",))})
ds_mean = ds_mean.assign_coords({"geolon_c": ds.geolon_c, "geolat_c": ds.geolat_c})

## Create the z-coordinate grid

In [9]:
coords={
    'X': {'center': 'xh',  'outer': 'xq'},
    'Y': {'center': 'yh',  'outer': 'yq'},
    'Z': {'center': 'z_l', 'outer': 'z_i'},
}
metrics = {
    ('X', 'Y'): ['areacello']
}
grid_mean = xgcm.Grid(
    ds_mean,
    coords=coords,
    metrics=metrics,
    boundary={"X":"periodic", "Y":"extend", "Z":"extend"},
    autoparse_metadata=False
)

### Save vertically-integrated mass budget diagnostics on the native horizontal grid

In [10]:
ds_integral = ds_mean[["umo", "vmo", "dhdt", "dynamics_h_tendency", "vert_remap_h_tendency", "boundary_forcing_h_tendency"]]
ds_integral = ds_integral.assign_coords(ds_mean.coords)
ds_integral = ds_integral.sum("z_l", keep_attrs=True).drop_dims(["z_i"]).compute()

In [11]:
ds_integral.attrs = {"description": "An example netcdf file containing all variables required to close the vertically-integrated mass budgets in MOM6. All variables are provided on the higher resolution (eddy-permitting) native grid."}

In [12]:
ds_integral.to_netcdf("../data/MOM6_global_example_barotropic_mass_budget.nc", mode="w")

### Save horizontally-coarsened diagnostics on all depth levels

In [13]:
ds_coarse = CM4Xutils.coarsen.horizontally_coarsen(ds_mean, grid_mean, {"X":6, "Y":6})

Skipping variable time_bnds because `cell_methods` attribute not defined.
Skipping variable areacello because `cell_methods` attribute not defined.
Skipping variable deptho because `cell_methods` attribute not defined.
Skipping variable wet because `cell_methods` attribute not defined.
Skipping variable wet_u because `cell_methods` attribute not defined.
Skipping variable wet_v because `cell_methods` attribute not defined.
Skipping variable dxCv because `cell_methods` attribute not defined.
Skipping variable dyCu because `cell_methods` attribute not defined.


In [14]:
ds_coarse.attrs = {"description": "An example netcdf file containing all variables required to close mass, heat, salt, and thermodynamic density budgets in MOM6, as well as some other useful variables. All variables have been conservatively coarsened from the higher resolution (eddy-permitting) native grid."}

In [15]:
ds_coarse.to_netcdf("../data/MOM6_global_example_diagnostics_zlevels.nc", mode="w")